# Matrix Decompositions

## Decompositions Related to Solving Linear Systems
A fundamental problem in linear algebra is solving the linear system $Ax=b$, if it is even possible. In this section, we will implement two versions of the LU algorithm and demonstrate how to use numpy to run other decomposition methods.

### LU Decomposition (Version 1)

**Pseudocode for LU_decomposition()**
1. m, n = shape(A) - *get the shape of A*
2. U = copy(A) - *create a copy of A using np.copy()*
3. L = $I_{m}$ - *initialize L with the mxm identity matrix*
4. for $j$ from $0$ to $n-1$: - iterate over the columns of A
    1. for $i$ from $j+1$ to $m-1$:
        1. $L_{i,j} \leftarrow U_{i,j}/U{j,j}$
        2. $U_{i,j:} \leftarrow U_{i,j:} - L_{i,j}U_{j,j:}$
5. Return L, U

<div class="alert alert-success">
Exercise:
Implement the first version of the LU decomposition.
</div>

In [2]:
import numpy as np

In [3]:
# generate a sample A that we can work on
A = np.random.randn(4, 3)
A

array([[ 0.28755976,  0.26935347, -0.26225958],
       [ 0.48785367, -0.73763766, -0.3108648 ],
       [ 0.55285262, -0.15319234,  0.74227847],
       [-1.65826549,  0.87885861, -0.43820351]])

In [26]:
def LU_decomposition(A):
    m, n = A.shape
    U = A.copy()
    L = np.eye(m)

    for j in range(n):
        for i in range(j+1, m):
            L[i, j] = U[i, j] / U[j, j]
            U[i, j:] = U[i, j:] - L[i, j]*U[j, j:]

    return L, U

In [40]:
L, U = LU_decomposition(A)
np.allclose(L.dot(U), A)

True

In [29]:
np.sqrt(((L.dot(U) - A)**2).sum())

5.095246377785861e-16

### Fast LU Decomposition via *Vectorization*
In our first LU Decomposition algorithm, notice that we had a loop inside a loop. One way to speed up the computation is by the use of **vectorization**. Simply put, vectorization means taking advantage of vectors to perform parallel computations instead of iterating over each entry in the vector.

In [37]:
def LU_decomposition2(A):
    m, n = A.shape
    U = A.copy()
    L = np.eye(m)

    for j in range(n):
        L[j+1:, j] = U[j+1:, j] / U[j, j]
        U[j+1:, j:] = U[j+1:, j:] - np.outer( L[j+1:, j], U[j, j:].T)

    return L, U

#### Why outer product?

<img src='./notebooks/sketch.jpg' width=700>

In [41]:
L1, U1 = LU_decomposition2(A)
np.allclose(L.dot(U), A)

True

In [39]:
np.sqrt(((L.dot(U) - A)**2).sum())

5.095246377785861e-16

In [43]:
np.all(L == L1)

True

In [44]:
np.all(U == U1)

True

In [59]:
# generate a sample A that we can work on
A = np.random.randn(100, 100)
A.shape

(100, 100)

In [60]:
%%timeit
L, U = LU_decomposition(A)

36 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%%timeit
L1, U1 = LU_decomposition2(A)

5.1 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Scipy Matrix Decomposition and Linear System Solvers

In [53]:
from scipy import linalg

#### LU Decomposition

In [62]:
%%timeit
lu, piv = linalg.lu_factor(A)

168 µs ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [71]:
# Generate a random matrix and a random vector
A = np.random.random((10, 10))
b = np.random.random(10)

# Compute the LU decomposition of A and Pivots
lu, piv = linalg.lu_factor(A)

# Find the solution x to Ax = b
x = linalg.lu_solve((lu, piv), b)
x

array([-1.62391468,  0.48156698,  1.33514319, -1.67806577,  1.51046784,
        1.84171878, -2.44578966,  0.25345874,  1.40257597, -2.0575119 ])

In [73]:
np.allclose(A.dot(x), b)

True

### QR Decomposition

In [76]:
# get values for Q and R
Q, R = linalg.qr(A)

# create new variable y
y = Q.T.dot(b)

# Solve the equivalent problem to Ax = b
x = linalg.solve(R, y) # Solve Rx = y

np.allclose(A.dot(x), b)

True

### Cholesky Decomposition

In [77]:
A = np.array([[9, 3, 1, 5],
              [3, 7, 5, 1],
              [1, 5, 9, 2],
              [5, 1, 2, 6]])

b = np.array([1, 1, 1, 1])

c, low = linalg.cho_factor(A)
x = linalg.cho_solve((c, low), b)
x

array([-0.01749271,  0.11953353,  0.01166181,  0.1574344 ])

## Other Linear Algebra Routines in Scipy

```Python
det() - determinant of a square matrix
eig() - eigenvalues and eigenvectors of a square matrix
inv() - inverse of an invertible matrix
norm() - norm of a matrix or vector
```

In [80]:
linalg.det(A)

686.0

In [81]:
linalg.eigh(A)

(array([ 1.11399545,  3.92957693,  9.55408093, 16.40234669]),
 array([[ 0.50652672, -0.29233034,  0.59370792,  0.55270658],
        [-0.49848052, -0.61982187, -0.34427146,  0.49881378],
        [ 0.39880928,  0.43995116, -0.61099505,  0.52352573],
        [-0.57956795,  0.58035059,  0.39455433,  0.41427173]]))

In [83]:
linalg.inv(A)

array([[ 0.30758017, -0.1851312 ,  0.12827988, -0.26822157],
       [-0.1851312 ,  0.3483965 , -0.20991254,  0.16618076],
       [ 0.12827988, -0.20991254,  0.24781341, -0.15451895],
       [-0.26822157,  0.16618076, -0.15451895,  0.41399417]])

In [84]:
linalg.norm(A)

19.4164878389476